In [ ]:
!pip install qiskit qiskit-ibm-runtime pylatexenc qiskit_aer

In [ ]:
import qiskit
import numpy as np
qiskit.version.get_version_info()

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np
# f(x) = 0 혹은 f(x) = 1 인 상수 함수의 도이치-조사 오라클 회로를 구현하는 함수
def dj_oracle_const(n):
    qy = QuantumRegister(1, 'y')
    qx = QuantumRegister(n, 'x')
    qc = QuantumCircuit(qy, qx) # qc 라는 이름의 n+1큐비트 양자 회로 객체를 생성

    np.random.seed()
    x = np.random.randint(2)
    if x == 1 :                 # if x==1 f(x)=1, if x==0 f(x)=0
      qc.x(0)
    qc.name = 'DJ-const'
    return qc              # qc 객체를 반환

circuit = dj_oracle_const(3) # 함수를 호출하여 회로를 표현하는 객체를 생성
circuit.draw(output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
# 균형 함수의 도이치-조사 오라클 회로를 구현하는 함수
def dj_oracle_balanced(n):
    qy = QuantumRegister(1, 'y')
    qx = QuantumRegister(n, 'x')
    qc = QuantumCircuit(qy, qx) # qc 라는 이름의 n+1큐비트 양자 회로 객체를 생성
    qc.x(qy)                # qy 에 X gate -> qy 상태가 |1> 이 됨
    qc.h(qy)                # qy 에 H gate -> qy 상태가 |-> 이 됨

    np.random.seed()
    s = np.random.randint(2,size=n) # using a random n-bit binary list s
    s[n-1] = 1                      # make sure s is not all 0's
    print(s)
    for i in range(n) :
      if s[i] == 1 :
        qc.cx(qx[i],qy)         # 제어 큐비트 qx[i], 타겟 큐비트 qy 인 CX(CNOT) gate

    qc.name = 'DJ-balanced'
    return qc               # qc 객체를 반환

circuit = dj_oracle_balanced(3) # 함수를 호출하여 회로를 표현하는 객체를 생성
circuit.draw(output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
# 임의의 n-qubit 도이치-조사 오라클 회로를 구현하는 함수
def dj_oracle(n):
    qy = QuantumRegister(1, 'y')
    qx = QuantumRegister(n, 'x')
    qc = QuantumCircuit(qy, qx) # qc 라는 이름의 n+1큐비트 양자 회로 객체를 생성
    np.random.seed()
    condition = np.random.choice(['constant','balanced'])
    print(condition)

    if condition == 'constant' :  # construct a random constant f(x)
      x = np.random.randint(2)
      print(x)
      if x == 1 :                 # if x==1 f(x)=1, if x==0 f(x)=0
        qc.x(qy)
    else :                        # construct a random balanced f(x)
      k = np.random.randint(1,n+1)   # make sure s is not all 0's
      a = np.random.permutation(n)
      a = a[:k]                    # a = k개의 n 보다 작은 정수들의 집합
      print(a)
      for x in a :
        qc.cx(qx[x],qy)         # 제어 큐비트 qx[x], 타겟 큐비트 qy 인 CX(CNOT) gate

    qc.name = 'DJ-Oracle'
    return qc              # qc 객체를 반환

circuit = dj_oracle(3) # 함수를 호출하여 회로를 표현하는 객체를 생성
circuit.draw(output='mpl')

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
# 도이치-조사 오라클 함수를 호출하여 도이치-조사 회로를 circuit 이라는 이름의 객체로 구현
n = 3
qx = QuantumRegister(n,'x')  # n 큐비트의 양자 레지스터 객체를 qx 라는 이름으로 생성, 회로에는 'x'로 표시
qy = QuantumRegister(1,'y')  # 1 큐비트의 양자 레지스터 객체를 qy 라는 이름으로 생성, 회로에는 'y'로 표시
c = ClassicalRegister(n,'c') # n 비트의 고전 레지스터 객체를 c 라는 이름으로 생성, 회로에는 'c'로 표시
circuit = QuantumCircuit(qy, qx, c) # qy,qx,c 를 가지는 양자 회로를 circuit 이라는 이름의 객체로 생성
circuit.x(qy)                # qy 큐비트에 X gate -> qy 큐비트 상태가 |1> 이 됨
circuit.h(qy)                # qy 큐비트에 H gate -> qy 큐비트 상태가 |-> 이 됨
circuit.h(qx)                # qx 의 모든 큐비트에 H gate 적용
circuit.barrier()
# dj_oracle_circuit() 을 호출하여 오라클 회로를 circuit 에 추가
circuit.compose(dj_oracle(n), range(n+1), inplace=True)
circuit.barrier()
circuit.h(qx)              # qx 의 모든 큐비트에 H gate 적용
circuit.measure(qx,c)      # qx 레지스터의 모든 큐비트들을 측정하여 c 레지스터로 읽는 회로를 구현
circuit.draw(output='mpl') # circuit 회로를 matplot library 형식의 그림 화일로 출력

In [ ]:
# AerSimulator 를 이용하여 circuit 회로를 측정한 결과를 출력
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)

isa_circuit = pm.run(circuit)
sampler = Sampler(mode=aer_sim)
job = sampler.run([isa_circuit], shots=1)
result = job.result()
count = result[0].data.c.get_counts()
print (count)

In [ ]:
n_zeros = n*'0'

if (n_zeros in count) :
  answer = 'constant'
else :
  answer = 'balanced'
print(f'f(x) is a {answer} function.')
